# Phi Native Implementation Demo

This notebook demonstrates the native Python implementation of integrated information (phi) using pyspi functions directly.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pyspi.calculator import Calculator
from pyspi.lib.phi_native import phi_comp
import tempfile
import yaml
import time

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Direct phi_native Function Usage

In [2]:
# Generate test data
np.random.seed(42)
data = np.random.randn(2, 100)  # 2 variables, 100 time points

# Set up parameters for phi computation
Z = np.array([1, 2])  # Partition
params = {"tau": 1}

# Test different phi configurations
configs = [
    {"type_of_phi": "star", "normalization": 0},
    {"type_of_phi": "star", "normalization": 1},
    {"type_of_phi": "Geo", "normalization": 0},
    {"type_of_phi": "Geo", "normalization": 1}
]

print("Direct phi_native function results:")
print("=" * 50)

for i, config in enumerate(configs):
    options = {**config, "type_of_dist": "Gauss"}
    
    start_time = time.time()
    phi_value = phi_comp(data, Z, params, options)
    end_time = time.time()
    
    print(f"Config {i+1}: phi_{config['type_of_phi']} (norm={config['normalization']})")
    print(f"  Value: {phi_value:.6f}")
    print(f"  Time: {(end_time-start_time)*1000:.2f} ms")
    print()

Direct phi_native function results:
Config 1: phi_star (norm=0)
  Value: 0.012398
  Time: 3.07 ms

Config 2: phi_star (norm=1)
  Value: 0.009339
  Time: 3.38 ms

Config 3: phi_Geo (norm=0)
  Value: 0.010218
  Time: 3.54 ms

Config 4: phi_Geo (norm=1)
  Value: 0.007697
  Time: 3.25 ms



## 2. Calculator with Configuration

In [3]:
# Create phi configuration
config = {
    '.statistics.infotheory': {
        'IntegratedInformation': {
            'labels': ['undirected', 'nonlinear', 'unsigned', 'bivariate', 'time-dependent'],
            'configs': [
                {'phitype': 'star'},
                {'phitype': 'star', 'normalization': 1},
                {'phitype': 'Geo'},
                {'phitype': 'Geo', 'normalization': 1}
            ]
        }
    }
}

# Create temporary config file
with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
    yaml.dump(config, f)
    config_path = f.name

# Use Calculator with configuration
np.random.seed(42)
test_data = np.random.randn(3, 200)  # 3 variables, 200 time points

print("Creating Calculator with phi configuration...")
calc = Calculator(test_data, configfile=config_path)

print(f"Calculator initialized with {len(calc.spis)} SPI configurations")

# Compute phi values
print("\nComputing phi values...")
start_time = time.time()
calc.compute()
end_time = time.time()

print(f"Computation completed in {end_time - start_time:.4f} seconds")
print(f"Number of result tables: {len(calc.table)}")

# Clean up
import os
os.unlink(config_path)

Creating Calculator with phi configuration...
Checking if optional dependencies exist...
Starting JVM with java class /Users/erron_lipenglin/Desktop/2025_Informational_str/20250916_github_phi/pyspi/pyspi/lib/jidt/infodynamics.jar.
Loading configuration file: /var/folders/rk/dhk7mvtn7vjgbrp3ytb077tw0000gn/T/tmpl9d0j2z9.yaml
*** Importing module .statistics.infotheory
[0] Adding SPI .statistics.infotheory.IntegratedInformation(x,y,{'phitype': 'star'})
Succesfully initialised SPI with identifier "phi_star_t-1_norm-0" and labels ['linear', 'unsigned', 'infotheory', 'temporal', 'undirected']
[1] Adding SPI .statistics.infotheory.IntegratedInformation(x,y,{'normalization': 1, 'phitype': 'star'})
Succesfully initialised SPI with identifier "phi_star_t-1_norm-1" and labels ['linear', 'unsigned', 'infotheory', 'temporal', 'undirected']
[2] Adding SPI .statistics.infotheory.IntegratedInformation(x,y,{'phitype': 'Geo'})
Succesfully initialised SPI with identifier "phi_Geo_t-1_norm-0" and labels [

Processing [None: phi_Geo_t-1_norm-1]: 100%|██████████| 4/4 [00:00<00:00, 31.99it/s]



Calculation complete. Time taken: 0.1274s

SPI Computation Results Summary

Total number of SPIs attempted: 4
Number of SPIs successfully computed: 4 (100.00%)
------------------------------------------------------------
Category       | Count | Percentage
------------------------------------------------------------
Successful     |     4 | 100.00%
NaNs           |     0 |   0.00%
Partial NaNs   |     0 |   0.00%
------------------------------------------------------------
Computation completed in 0.1822 seconds
Number of result tables: 3


## 3. Results Visualization

In [7]:
# Display results
print("Phi computation results:")
print("=" * 60)

for table_name, table_data in calc.table.items():
    print(f"\n{table_name}:")
    print("-" * 40)
    print(table_data)

    # Show statistics - handle both Series and DataFrame
    if hasattr(table_data, 'select_dtypes'):
        # DataFrame case
        numeric_data = table_data.select_dtypes(include=[np.number])
        if not numeric_data.empty:
            finite_values = numeric_data.values[np.isfinite(numeric_data.values)]
        else:
            finite_values = []
    else:
        # Series case
        if np.issubdtype(table_data.dtype, np.number):
            finite_values = table_data.values[np.isfinite(table_data.values)]
        else:
            finite_values = []

    if len(finite_values) > 0:
        print(f"  Range: [{finite_values.min():.6f}, {finite_values.max():.6f}]")
        print(f"  Mean: {finite_values.mean():.6f}")
        print(f"  Std: {finite_values.std():.6f}")

Phi computation results:

('phi_star_t-1_norm-0', 'proc-0'):
----------------------------------------
proc-0         NaN
proc-1    0.003826
proc-2    0.010770
Name: (phi_star_t-1_norm-0, proc-0), dtype: float64
  Range: [0.003826, 0.010770]
  Mean: 0.007298
  Std: 0.003472

('phi_star_t-1_norm-0', 'proc-1'):
----------------------------------------
proc-0    0.003826
proc-1         NaN
proc-2    0.005747
Name: (phi_star_t-1_norm-0, proc-1), dtype: float64
  Range: [0.003826, 0.005747]
  Mean: 0.004787
  Std: 0.000960

('phi_star_t-1_norm-0', 'proc-2'):
----------------------------------------
proc-0    0.010770
proc-1    0.005747
proc-2         NaN
Name: (phi_star_t-1_norm-0, proc-2), dtype: float64
  Range: [0.005747, 0.010770]
  Mean: 0.008259
  Std: 0.002512

('phi_star_t-1_norm-1', 'proc-0'):
----------------------------------------
proc-0         NaN
proc-1    0.002698
proc-2    0.007596
Name: (phi_star_t-1_norm-1, proc-0), dtype: float64
  Range: [0.002698, 0.007596]
  Mean: 0.00